In [1]:
import json
from tqdm import tqdm
import numpy as np
import pytrec_eval

In [12]:
preds = '/mnt/scratch/chenqu/stateful_search/46/test_preds.txt'
preds_1 = '/mnt/scratch/chenqu/stateful_search/97/test_preds.txt'
dev_file = '/mnt/scratch/chenqu/aol/preprocessed/session_dev_small.txt'
test_file = '/mnt/scratch/chenqu/aol/preprocessed/session_test.txt'
train_file = '/mnt/scratch/chenqu/aol/preprocessed/session_train.txt'

In [13]:
with open(preds) as fin:
    res = json.load(fin)
qrels = res['qrels']
run = res['run']

with open(preds_1) as fin:
    res_1 = json.load(fin)
qrels = res['qrels']
run = res['run']

In [4]:
evaluator = pytrec_eval.RelevanceEvaluator(qrels, {'recip_rank', 'ndcg', 'ndcg_cut'})
eval_res = evaluator.evaluate(run)
mrr_list = [v['recip_rank'] for v in eval_res.values()]
ndcg_list = [v['ndcg'] for v in eval_res.values()]
ndcg_1_list = [v['ndcg_cut_1'] for v in eval_res.values()]
ndcg_3_list = [v['ndcg_cut_3'] for v in eval_res.values()]
ndcg_10_list = [v['ndcg_cut_10'] for v in eval_res.values()]

print('mrr', np.average(mrr_list))
print('ndcg_list', np.average(ndcg_list))
print('ndcg_1_list', np.average(ndcg_1_list))
print('ndcg_3_list', np.average(ndcg_3_list))
print('ndcg_10_list', np.average(ndcg_10_list))


mrr 0.5365898366310046
ndcg_list 0.6344083508761332
ndcg_1_list 0.3787470948935779
ndcg_3_list 0.517421151530598
ndcg_10_list 0.5953927323134787


In [5]:
res.keys()

dict_keys(['qrels', 'run', 'ranker_test_all_label_ids', 'guids', 'preds'])

In [7]:
def mrr(preds, labels, doc_num_list):
    mrr_list = []
    for num in tqdm(doc_num_list):
        cur_preds = preds[: num]
        cur_labels = labels[: num]
        mrr_list.append(single_mrr(cur_preds, cur_labels))
        
        preds = preds[num :]
        labels = labels[num :]
        
    return {'mrr': np.average(mrr_list)}, mrr_list
        
def single_mrr(preds, labels):
    score = 0.0
    index_rank = np.argsort(preds)[::-1]
    for rank, i in enumerate(index_rank):
        if labels[i] == 1:
            score = 1.0 / (rank + 1.0)
            break
            
    return score

In [17]:
mrr(res['preds'][450:500], res['ranker_test_all_label_ids'][450:500], [50] * 1)

100%|██████████| 1/1 [00:00<00:00, 1213.28it/s]


({'mrr': 1.0}, [1.0])

In [18]:
res['preds'][450:500]

[0.9943580627441406,
 6.132764974609017e-05,
 6.156767631182447e-05,
 0.00018378469394519925,
 6.761841359548271e-05,
 6.253711035242304e-05,
 7.632232882315293e-05,
 6.156767631182447e-05,
 6.156767631182447e-05,
 6.156767631182447e-05,
 9.278285142499954e-05,
 6.132764974609017e-05,
 8.922864799387753e-05,
 6.156767631182447e-05,
 0.00010310213110642508,
 6.156767631182447e-05,
 6.156767631182447e-05,
 0.001796149299480021,
 6.278187356656417e-05,
 6.156767631182447e-05,
 0.0039917281828820705,
 6.278187356656417e-05,
 6.205050158314407e-05,
 6.156767631182447e-05,
 6.132764974609017e-05,
 0.010189338587224483,
 6.108855450293049e-05,
 6.108855450293049e-05,
 6.132764974609017e-05,
 6.156767631182447e-05,
 6.132764974609017e-05,
 6.156767631182447e-05,
 0.0009436191758140922,
 6.205050158314407e-05,
 7.031182758510113e-05,
 6.253711035242304e-05,
 0.0005625758785754442,
 6.132764974609017e-05,
 6.18085905443877e-05,
 6.108855450293049e-05,
 6.156767631182447e-05,
 6.156767631182447e-

In [16]:
mrr(res_1['preds'][450:500], res_1['ranker_test_all_label_ids'][450:500], [50] * 1)

100%|██████████| 1/1 [00:00<00:00, 1332.79it/s]


({'mrr': 0.3333333333333333}, [0.3333333333333333])

In [19]:
res_1['preds'][450:500]

[0.08288274705410004,
 0.019643759354948997,
 0.0008426422718912363,
 0.7945444583892822,
 0.011892049573361874,
 0.0010691260686144233,
 0.0010403183987364173,
 0.0008409997099079192,
 0.0009181902860291302,
 0.0008426422718912363,
 0.0008492455235682428,
 0.0008592479862272739,
 0.0008525669109076262,
 0.0008409997099079192,
 0.0008727670647203922,
 0.0008475900976918638,
 0.0008393599418923259,
 0.0015947327483445406,
 0.0009075026027858257,
 0.0008409997099079192,
 0.0068107303231954575,
 0.0008525669109076262,
 0.0019877739250659943,
 0.0010649629402905703,
 0.0008575725951232016,
 0.07871317118406296,
 0.0010628870222717524,
 0.002878348110243678,
 0.0008393599418923259,
 0.06881856918334961,
 0.0008969392511062324,
 0.001242417492903769,
 0.2045801728963852,
 0.0008426422718912363,
 0.0016711372882127762,
 0.006488270126283169,
 0.0008592479862272739,
 0.0008847708231769502,
 0.021145397797226906,
 0.000845937873236835,
 0.031618762761354446,
 0.0008393599418923259,
 0.001257044